In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

/home/sumit/anaconda3/envs/torch_gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 30
batch_size = 100
learning_rate = 0.001

# Image processing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data/',
                                           train = True,
                                           transform = transform,
                                           download = True)
test_dataset = torchvision.datasets.CIFAR10(root = './data/',
                                           train = False,
                                           transform = transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                     batch_size = batch_size,
                                     shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                    batch_size = batch_size,
                                    shuffle = False)

Files already downloaded and verified


In [4]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [5]:
class ResNet(nn.Module):
    def __init__(self,block,layers,num_classes = 10):
        """layers will be a list : [2,2,2]
        """
        super(ResNet,self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3,16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        # Layers means numbers of residual block in each layer
        self.layer1 = self.make_layer(block,16,layers[0])
        self.layer2 = self.make_layer(block,32,layers[1],stride = 2)
        self.layer3 = self.make_layer(block,64,layers[2],stride = 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64,num_classes)
        
    def make_layer(self,block,out_channels,blocks,stride = 1):
        """Blocks means number of residual block in each layer"""
        downsample = None
        if (self.in_channels != out_channels) or (stride!=1):
            downsample = nn.Sequential(conv3x3(self.in_channels,out_channels,stride = stride),
                                        nn.BatchNorm2d(out_channels))
        residual_blocks = []
        residual_blocks.append(block(self.in_channels,out_channels,stride,downsample = downsample))
        self.in_channels = out_channels
        for i in range(1,blocks):
            residual_blocks.append(block(out_channels,out_channels))
        
        return nn.Sequential(*residual_blocks)
    
    def forward(self,x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        
        return out

In [6]:
model = ResNet(ResidualBlock,[2,2,2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)


In [7]:
decay = 0
for epoch in range(num_epochs):
    # Decay the learning rate by a factor of 0.5 evey 20 epochs
    if (epoch + 1)%20 == 0:
        decay += 1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print(f"The new learning rate is {optimizer.param_groups[0]['lr']}")
        
        
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/30], Step [100/500] Loss: 1.5536
Epoch [1/30], Step [200/500] Loss: 1.6498
Epoch [1/30], Step [300/500] Loss: 1.4049
Epoch [1/30], Step [400/500] Loss: 1.3079
Epoch [1/30], Step [500/500] Loss: 1.0469
Epoch [2/30], Step [100/500] Loss: 1.1029
Epoch [2/30], Step [200/500] Loss: 1.0475
Epoch [2/30], Step [300/500] Loss: 1.0140
Epoch [2/30], Step [400/500] Loss: 0.9359
Epoch [2/30], Step [500/500] Loss: 0.9944
Epoch [3/30], Step [100/500] Loss: 0.8778
Epoch [3/30], Step [200/500] Loss: 0.7871
Epoch [3/30], Step [300/500] Loss: 0.6068
Epoch [3/30], Step [400/500] Loss: 0.9264
Epoch [3/30], Step [500/500] Loss: 0.8594
Epoch [4/30], Step [100/500] Loss: 0.8654
Epoch [4/30], Step [200/500] Loss: 0.9612
Epoch [4/30], Step [300/500] Loss: 0.7939
Epoch [4/30], Step [400/500] Loss: 0.9033
Epoch [4/30], Step [500/500] Loss: 0.9307
Epoch [5/30], Step [100/500] Loss: 0.9351
Epoch [5/30], Step [200/500] Loss: 0.6297
Epoch [5/30], Step [300/500] Loss: 0.6522
Epoch [5/30], Step [400/500] Loss:

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of model on the test images: {100*correct/total}')

Accuracy of model on the test images: 86.78
